### Q1. Notebook

In [4]:
## Check sklearn version
!pip freeze | grep scikit-learn

In [4]:
## Check Python Version
!python -V

Python 3.11.2


In [5]:
## Load Libraries
import pickle
import pandas as pd

In [46]:
!pip freeze | grep pandas

pandas==2.2.2


In [1]:
! pip list

Package                            Version
---------------------------------- -----------
alembic                            1.13.1
aniso8601                          9.0.1
asttokens                          2.4.1
blinker                            1.8.2
cachetools                         5.3.3
certifi                            2024.2.2
charset-normalizer                 3.3.2
click                              8.1.7
cloudpickle                        3.0.0
colorama                           0.4.6
comm                               0.2.2
contourpy                          1.2.1
cycler                             0.12.1
debugpy                            1.8.1
decorator                          5.1.1
Deprecated                         1.2.14
distlib                            0.3.8
docker                             7.1.0
entrypoints                        0.4
executing                          2.0.1
filelock                           3.14.0
Flask                              3.0.3
fon


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
!ls .

starter.ipynb


In [10]:
with open('../models/model.bin', 'rb') as f:
    dv, model = pickle.load(f)

In [11]:
## Handle Duration
categorical = ["PULocationID", "DOLocationID"]

def read_data(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [23]:
## Load Mar Data'23
df = read_data("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet")

In [14]:
## Isolating stated catgeories, call dict vectorizer and predict
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [19]:
print(f"Q1. Standard Deviation of predictions: {round(y_pred.std(), 3)}")

Q1. Standard Deviation of predictions: 6.247


### Q2. Preparing the output

In [26]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration,ride_id
0,2,2023-03-01 00:06:43,2023-03-01 00:16:43,1.0,0.00,1.0,N,238,42,2,...,1.0,0.5,0.00,0.0,1.0,11.10,0.0,0.00,10.000000,2023/03_0
1,2,2023-03-01 00:08:25,2023-03-01 00:39:30,2.0,12.40,1.0,N,138,231,1,...,6.0,0.5,12.54,0.0,1.0,76.49,2.5,1.25,31.083333,2023/03_1
2,1,2023-03-01 00:15:04,2023-03-01 00:29:26,0.0,3.30,1.0,N,140,186,1,...,3.5,0.5,4.65,0.0,1.0,28.05,2.5,0.00,14.366667,2023/03_2
3,1,2023-03-01 00:49:37,2023-03-01 01:01:05,1.0,2.90,1.0,N,140,43,1,...,3.5,0.5,4.10,0.0,1.0,24.70,2.5,0.00,11.466667,2023/03_3
4,2,2023-03-01 00:08:04,2023-03-01 00:11:06,1.0,1.23,1.0,N,79,137,1,...,1.0,0.5,2.44,0.0,1.0,14.64,2.5,0.00,3.033333,2023/03_4


In [25]:
## Ride ID Column
year = 2023
month = 3

# df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])

# df["year"] = df["tpep_pickup_datetime"].dt.year
# df["month"] = df["tpep_pickup_datetime"].dt.month

df["ride_id"] = f"{year:04d}/{month:02d}_" + df.index.astype("str")

In [33]:
## Save predictions to dataframe
df_results = pd.DataFrame(columns = ["predictions", "ride_id"])
df_results["predictions"] = y_pred
df_results["ride_id"] = df["ride_id"]

df_results.head()

,predictions,ride_id
0,16.245906,2023/03_0
1,26.134796,2023/03_1
2,11.884264,2023/03_2
3,11.997720,2023/03_3
4,10.234486,2023/03_4


In [40]:
## Path  to output file
import os
from datetime import date

output_pth = "../output_files"
if not os.path.exists(output_pth):
    os.makedirs(output_pth)

output_file = output_pth + "/results_" + str(date.today()) + ".parquet"

df_results.to_parquet(
    output_file,
    engine = 'pyarrow',
    compression = None,
    index = False
)

In [43]:
# Size of the output file

! ls -lh ../output_files/* | awk '{print $5, $9}'
# $5, $9: These represent the 5th and 9th fields of each line.
# (i.e., in kilobytes (K), megabytes (M), gigabytes (G), etc.)

65M ../output_files/results_2024-06-08.parquet


### Q3. Creating the scoring script

In [44]:
!jupyter nbconvert --to script starter.ipynb

[NbConvertApp] Converting notebook starter.ipynb to script
[NbConvertApp] Writing 2528 bytes to starter.py


In [ ]:
#> pip install pipenv
#> pipenv --python 3.11
#> pipenv install scikit-learn==1.5.0
#"scikit-learn": {
            # "hashes": [
            #     "sha256:057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c",